In [25]:
import numpy as np
import pandas as pd
import os
import pdb
from random import random
import math
import matplotlib.pyplot as plta
import pylab
import matplotlib.animation as animation
from scipy.spatial import distance
from scipy.spatial.distance import euclidean
from mpl_toolkits.mplot3d import Axes3D
def distance(p0, p1):
    return math.sqrt((p0[0] - p1[0])**2 + (p0[1] - p1[1])**2 +(p0[2] - p1[2])**2  )
def rotate(origin, point, angle):
    """
    Rotate a point counterclockwise by a given angle around a given origin.

    The angle should be given in radians.
    """
    ox, oy = origin
    px, py = point

    qx = ox + math.cos(angle) * (px - ox) - math.sin(angle) * (py - oy)
    qy = oy + math.sin(angle) * (px - ox) + math.cos(angle) * (py - oy)
    return qx, qy
def count_xtal_waters(ROI, df, radius):
    ref_waters_in_OEC = []
    OEC_ROI = radius
    for n,rew in df.iterrows():
        point1= [float(df.loc[n,'x']),float(df.loc[n,'y']),float(df.loc[n,'z'])]
        point2= [float(ROI['x'].values[0]),float(ROI['y'].values[0]),float(ROI['z'].values[0])]
        if ((distance(point1,point2) < OEC_ROI )):
            ref_waters_in_OEC.append(df.loc[n])
    return len(ref_waters_in_OEC)

# Open up solvent peaks file

In [26]:
colspecs_pdb = [(0, 6), (6, 11), (12, 16), (16, 17), (17, 20), (21, 22), (22, 26),
            (26, 27), (30, 38), (38, 46), (46, 54), (54, 60), (60, 66), (76, 78),
            (78, 80)]

names_pdb = ['ATOM', 'serial', 'name', 'altloc', 'resname', 'chainid', 'resseq',
         'icode', 'x', 'y', 'z', 'occupancy', 'tempfactor', 'element', 'charge']

pdb_path = 'files/peaks_SOL_rms2.69_7_13_22.pdb'
peaks = pd.read_fwf(pdb_path, names=names_pdb, colspecs=colspecs_pdb,skiprows=4)

In [27]:
peaks_in_roi = peaks.copy()
peaks

,ATOM,serial,name,altloc,resname,chainid,resseq,icode,x,y,z,occupancy,tempfactor,element,charge
0,ATOM,2327.0,O,NaN,HOH,X,1.0,NaN,81.653,108.127,20.046,10.81,10.81,O,NaN
1,ATOM,6062.0,O,NaN,HOH,X,2.0,NaN,84.839,43.055,55.808,10.62,10.62,O,NaN
2,ATOM,6398.0,O,NaN,HOH,X,3.0,NaN,93.976,20.540,58.748,10.54,10.54,O,NaN
3,ATOM,5787.0,O,NaN,HOH,X,4.0,NaN,80.996,33.221,53.062,10.50,10.50,O,NaN
4,ATOM,4618.0,O,NaN,HOH,X,5.0,NaN,28.257,175.076,39.768,10.49,10.49,O,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8282,ATOM,3153.0,O,NaN,HOH,X,8283.0,NaN,6.840,218.940,26.383,2.69,2.69,O,NaN
8283,ATOM,2979.0,O,NaN,HOH,X,8284.0,NaN,97.727,143.706,25.266,2.69,2.69,O,NaN
8284,ATOM,979.0,O,NaN,HOH,X,8285.0,NaN,67.252,163.799,7.752,2.69,2.69,O,NaN
8285,ATOM,2746.0,O,NaN,HOH,X,8286.0,NaN,62.696,159.396,23.082,2.69,2.69,O,NaN


## Grab xtal waters

In [28]:
gro_path = 'files/0F.pdb_fittedto55all.pdb' #Note that 0F structrue has extended chain G relative to LU34_0F
reference = pd.read_fwf(gro_path, names=names_pdb, colspecs=colspecs_pdb, skiprows=4)
ref2=reference[(reference['resname']=='OOO') | (reference['resname']=='HOH') | (reference['name']=='W1')|(reference['name']=='W2')|(reference['name']=='W3')|(reference['name']=='W4')]
df_water =ref2
df_water.loc[2627,'resseq']='W1'
df_water.loc[2628,'resseq']='W2'
df_water.loc[2629,'resseq']='W3'
df_water.loc[2630,'resseq']='W4'
df_water

/Users/yyk_lab/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,ATOM,serial,name,altloc,resname,chainid,resseq,icode,x,y,z,occupancy,tempfactor,element,charge
2627,HETATM,2628.0,W1,NaN,OEC,A,W1,NaN,83.512,35.203,52.954,1.00,27.44,O,NaN
2628,HETATM,2629.0,W2,NaN,OEC,A,W2,NaN,81.644,37.556,53.164,1.00,29.43,O,NaN
2629,HETATM,2630.0,W3,NaN,OEC,A,W3,NaN,83.046,40.226,53.709,1.00,28.51,O,NaN
2630,HETATM,2631.0,W4,NaN,OEC,A,W4,NaN,85.200,42.041,55.104,1.00,31.98,O,NaN
16552,HETATM,16547.0,O,NaN,OOO,G,1.0,NaN,65.040,58.188,50.009,1.00,24.85,O,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52252,HETATM,52213.0,O,NaN,HOH,S,1796.0,NaN,22.830,59.185,-0.104,1.00,57.12,O,NaN
52253,HETATM,52214.0,O,NaN,HOH,S,1797.0,NaN,93.855,74.540,19.748,0.98,61.32,O,NaN
52254,HETATM,52215.0,O,NaN,HOH,S,1798.0,NaN,21.829,75.450,31.332,1.00,72.32,O,NaN
52255,HETATM,52216.0,O,NaN,HOH,S,1799.0,NaN,22.787,-59.484,21.536,1.00,65.81,O,NaN


## Define ROI

In [30]:
radius=25
ROI_OEC1 = reference[(reference['resname'] =='OEC') & (reference['chainid'] =='A')& (reference['name'] =='O1')]


# Select ROI for calculation 

In [31]:
ROI = ROI_OEC1

# Make list of XTAL waters found in ROI

In [32]:
ref_waters_in_OEC = []
    
for n,rew in df_water.iterrows():
    point1= [float(df_water.loc[n,'x']),float(df_water.loc[n,'y']),float(df_water.loc[n,'z'])]
  
    for x in range(0,len(ROI)):
        point2= [float(ROI['x'].values[x]),float(ROI['y'].values[x]),float(ROI['z'].values[x])]
        if ((distance(point1,point2) < radius )):
            ref_waters_in_OEC.append(df_water.loc[n])
            break

ref_waters_in_OEC=pd.DataFrame(ref_waters_in_OEC)
print(len(ref_waters_in_OEC))

222


# Make list of peaks in ROI

In [33]:
peaks_in_OEC=[]
for z,ruw in peaks_in_roi.iterrows():
    point1= [float(peaks_in_roi.loc[z,'x']),float(peaks_in_roi.loc[z,'y']),float(peaks_in_roi.loc[z,'z'])]
    
    for x in range(0,len(ROI)):
        point2= [float(ROI['x'].values[x]),float(ROI['y'].values[x]),float(ROI['z'].values[x])]
        if ((distance(point1,point2) < radius) ):
            peaks_in_OEC.append(peaks_in_roi.loc[z])
            break
peaks_in_OEC=pd.DataFrame(peaks_in_OEC)
len(peaks_in_OEC)

338

## Find matches to MD peaks

In [34]:
matches= pd.DataFrame(columns=['XTAL ID', 'Closest MD Partner', 'Dist', 'Thresh'] ,dtype=object) #xtal waters
matches_all =pd.DataFrame(columns=['MD ID', 'Closest XTAL Partner', 'Dist', "Thresh"], dtype=object)  #md_peaks
dist_array=[]
partner_array=[]
strength_array=[]

peaks_array=peaks_in_OEC['resseq']
water_array=ref_waters_in_OEC['resseq']
chain_array=ref_waters_in_OEC['chainid']

for h,rhu in peaks_in_OEC.iterrows():
    point1= [float(peaks_in_OEC.loc[h,'x']),float(peaks_in_OEC.loc[h,'y']),float(peaks_in_OEC.loc[h,'z'])]
    dist=10000
    partner=0
    strength= peaks_in_OEC.loc[h,'occupancy']
    for z,r in ref_waters_in_OEC.iterrows():
        point2= [float(ref_waters_in_OEC.loc[z,'x']),float(ref_waters_in_OEC.loc[z,'y']),float(ref_waters_in_OEC.loc[z,'z'])]
        dist_try=distance(point1,point2)
        if dist_try <dist:
            dist=dist_try #update value
            partner= ref_waters_in_OEC.loc[z,'resseq']
    dist_array.append(dist)
    partner_array.append(partner)      
    strength_array.append(strength)
matches_all['MD ID'] = peaks_array
matches_all['Closest XTAL Partner'] = partner_array
matches_all['Dist'] = dist_array
matches_all['Thresh'] = strength_array



# Find matches to XTAL waters

In [35]:
dist_array=[]
partner_array=[]
strength_array=[]

for h,rhu in ref_waters_in_OEC.iterrows():
    point1= [float(ref_waters_in_OEC.loc[h,'x']),float(ref_waters_in_OEC.loc[h,'y']),float(ref_waters_in_OEC.loc[h,'z'])]
    dist=10000
    partner=0
    strength=0
    for z,r in peaks_in_OEC.iterrows():
        point2= [float(peaks_in_OEC.loc[z,'x']),float(peaks_in_OEC.loc[z,'y']),float(peaks_in_OEC.loc[z,'z'])]
        dist_try=distance(point1,point2)
        if dist_try <dist:
            dist=dist_try #update value
            partner= peaks_in_OEC.loc[z,'resseq']
            strength=peaks_in_OEC.loc[z,'occupancy']
         
    dist_array.append(dist)
    partner_array.append(partner)   
    strength_array.append(strength)     

matches['XTAL ID'] = water_array
matches['Closest MD Partner'] = partner_array
matches['Dist'] = dist_array
matches['Thresh'] = strength_array
matches['Chain']=chain_array

In [36]:
pr=pd.DataFrame(columns=('thresh', 'recall', 'precision', 'f1'), dtype=object) #precision/recall dataframe 0.5 dist
pr2=pd.DataFrame(columns=('thresh', 'recall', 'precision', 'f1'), dtype=object) #precision/recall dataframe 1.0 dist
pr3=pd.DataFrame(columns=('thresh', 'recall', 'precision', 'f1'), dtype=object) #precision/recall dataframe 1.4 dist
npr = 100 #number of precision/recall peaks we want 

df1=matches_all
df2=matches

pmax = np.max(df1['Thresh']) 
pmin = np.min(df1['Thresh']) 

df1= df1.sort_values(by=['Dist']) 
df2 = df2.sort_values(by=['Dist'])
df1_copy = df1.copy()
df2_copy = df2.copy()


# Getting agreement around active site 

In [37]:
df2_copy.to_pickle('files/Matches_XTALwater_OEC_chain1_2.69_rms_july_13' )

# Percent recall

In [38]:
len(df2_copy[(df2_copy['Dist']<1) ]) / len(df2_copy)

0.8513513513513513